In [ ]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

file = glob.glob("train/train.tsv")
pd.read_csv("train/train.tsv", delimiter='\t')
data_list = []
data_list.append(pd.read_csv("train/train.tsv", delimiter='\t', index_col=0))
df = pd.concat(data_list)
df.shape

In [5]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val = train_test_split(df, test_size=0.2)

col = "Horizontal_Distance_To_Hydrology"

train_y = df_train(col)
train_x = df_train.drop(col, axis=1)

val_y = df_val(col)
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)


TypeError: 'DataFrame' object is not callable